In [28]:
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import re
import time
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import numpy as np

In [2]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = wd.Chrome(executable_path = "C:\\Users\\JeSeok\\chromedriver.exe" ,options=chrome_options)

In [3]:
# 삼성 스마트폰 상품코드 모으기
# 1) 삼성 스마트폰 검색 상품 리스트 최대 페이지수
url = "https://www.amazon.com/-/ko/s?k=galaxy&i=mobile&rh=n%3A7072561011%2Cp_89%3ASamsung+Electronics%2Cp_n_availability%3A2661601011%2Cp_n_feature_nineteen_browse-bin%3A9521921011&dc&language=ko&qid=1639963921&rnid=9521918011&ref=sr_nr_p_n_feature_nineteen_browse-bin_3"
driver.get(url)
driver.implicitly_wait(10)

res = driver.page_source
obj = bs(res, 'html.parser')
source = driver.page_source
bs_obj = bs(source, "html.parser")

# 개발자 도구 내 정보가 수시로 바뀌기 때문에 웹페이지 확인 후 하드코딩 또는 개발자 도구 확인 후 사용
# 개발자 도구 확인 : max_page = int(bs_obj.find_all('span',{'class':'s-pagination-item s-pagination-disabled'})[0].get_text().strip())
# 홈페이지 확인 : max_page = 19
max_page = int(bs_obj.find_all('span',{'class':'s-pagination-item s-pagination-disabled'})[0].get_text().strip())

driver.close()
driver.quit()

In [4]:
# 삼성 스마트폰 상품코드 모으기
samsung_prod_cd = []
driver = wd.Chrome(executable_path = "C:\\Users\\JeSeok\\chromedriver.exe" ,options=chrome_options)
for i in range(1, max_page+1):
    url = "https://www.amazon.com/-/ko/s?k=galaxy&i=mobile&rh=n%3A7072561011%2Cp_89%3ASamsung+Electronics%2Cp_n_availability%3A2661601011%2Cp_n_feature_nineteen_browse-bin%3A9521921011&dc&language=ko&qid=1639963921&rnid=9521918011&ref=sr_nr_p_n_feature_nineteen_browse-bin_3&page={}".format(i)
    driver.get(url)
    driver.implicitly_wait(10)
    
    time.sleep(3)
    
    items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
    for item in items:
        data_asin = item.get_attribute("data-asin")
        samsung_prod_cd.append(data_asin)
        
driver.close()
driver.quit()

In [5]:
# 애플 아이폰 상품코드 모으기
# 1) 애플 아이폰 검색 상품 리스트 최대 페이지수
driver = wd.Chrome(executable_path = "C:\\Users\\JeSeok\\chromedriver.exe" ,options=chrome_options)
url = "https://www.amazon.com/-/ko/s?k=phone&i=mobile&bbn=7072561011&rh=n%3A7072561011%2Cp_89%3AApple%2Cp_n_feature_nineteen_browse-bin%3A9521921011%2Cp_n_availability%3A2661601011&s=relevancerank&dc&language=ko&qid=1640061794&rnid=2661599011&ref=sr_pg_1"
driver.get(url)
driver.implicitly_wait(10)

res = driver.page_source
obj = bs(res, 'html.parser')
source = driver.page_source
bs_obj = bs(source, "html.parser")

# 개발자 도구 내 정보가 수시로 바뀌기 때문에 웹페이지 확인 후 하드코딩 또는 개발자 도구 확인 후 사용
# 개발자 도구 확인 : max_page = int(bs_obj.find_all('a',{'class':'s-pagination-item s-pagination-button'})[3].get_text().strip())
# 홈페이지 확인 : max_page = 4
max_page = int(bs_obj.find_all('a',{'class':'s-pagination-item s-pagination-button'})[3].get_text().strip())

driver.close()
driver.quit()

In [6]:
# 애플 아이폰 상품코드 모으기
apple_prod_cd = []
driver = wd.Chrome(executable_path = "C:\\Users\\JeSeok\\chromedriver.exe" ,options=chrome_options)
for i in range(1, max_page+1):
    url = "https://www.amazon.com/-/ko/s?k=phone&i=mobile&bbn=2335752011&rh=n%3A7072561011%2Cp_89%3AApple%2Cp_n_availability%3A2661601011%2Cp_n_feature_nineteen_browse-bin%3A9521921011&s=relevancerank&dc&language=ko&qid=1639963835&rnid=9521918011&ref=sr_pg_1&page={}".format(i)
    driver.get(url)
    driver.implicitly_wait(10)
    
    time.sleep(3)
    
    items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
    for item in items:
        data_asin = item.get_attribute("data-asin")
        apple_prod_cd.append(data_asin)
        
driver.close()
driver.quit()

In [7]:
# 코드 중복 제거
samsung_prod_cd=list(set(samsung_prod_cd))
apple_prod_cd=list(set(apple_prod_cd))

In [8]:
# 각 코드 개수
print('samsung : ', len(samsung_prod_cd), ', apple : ', len(apple_prod_cd))

samsung :  454 , apple :  96


In [9]:
# 최종 CSV파일에 포함, 미포함 상품코드 목록
samsung_df_in_cd = []
samsung_df_out_cd = []
apple_df_in_cd = []
apple_df_out_cd = []

In [10]:
# 리뷰, 평점 데이터 스크래핑
def amazon_reviews(pr_cd):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument("--single-process")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    driver = wd.Chrome(executable_path = "C:\\Users\\JeSeok\\chromedriver.exe" ,options=chrome_options)
    
    url = "https://www.amazon.com/product-reviews/{}/ref=cm_cr_getr_d_paging_btm_next_8?ie=UTF8&reviewerType=%3Dall_reviews&sortBy=recent".format(pr_cd)
    driver.get(url)
    driver.implicitly_wait(10)
    
    res = driver.page_source
    obj = bs(res, 'html.parser')
    
    # ex) rev ='1,124 글로벌 평점 | 177 글로벌 리뷰' 
    # -> int(re.findall('\d+',rev)[1]) : 124 출력이기 때문에 ','을 빈값으로 설정
    rev = obj.select('#filter-info-section > div > span')[0].get_text().strip().replace(',','') 
    rev = int(re.findall('\d+',rev)[1])
    
    # 리뷰가 없는 경우 빈값 반환
    if rev==0:
        return ''

    products = [] # 제품명
    titles = [] # 리뷰 타이틀
    stars = [] # 리뷰 평점
    contents = [] # 리뷰 내용
    
    for page_num in range(1,int(np.ceil(rev/10))+1):
        url = "https://www.amazon.com/product-reviews/{}/ref=cm_cr_getr_d_paging_btm_next_8?ie=UTF8&reviewerType=%3Dall_reviews&sortBy=recent&pageNumber={}".format(pr_cd,page_num)
        driver.get(url)
        driver.implicitly_wait(10)
        source = driver.page_source
        bs_obj = bs(source, "html.parser")
        
        time.sleep(3)

        for i in bs_obj.findAll('a',{'data-hook':'review-title'}):
            titles.append(i.get_text().strip())
            
        for i in bs_obj.findAll('span', {'data-hook':'review-body'}):
            contents.append(i.get_text().strip())

        for i in bs_obj.findAll('i',{'data-hook':'review-star-rating'}):
            stars.append(int(i.get_text()[7]))
    
    # 미국 외 다른나라 리뷰만 있는 경우 빈값 반환
    if len(titles)==0:
        return ''
    elif len(contents)==0:
        return ''
    elif len(stars)==0:
        return ''
    
    for i in range(rev):
        products.append(bs_obj.find('a',{'data-hook':'product-link'}).get_text().strip())
    
    driver.close()
    driver.quit()
    
    products = pd.DataFrame(products)
    stars = pd.DataFrame(stars)
    titles = pd.DataFrame(titles)
    contents = pd.DataFrame(contents)
    
    df = pd.concat([products, stars, titles, contents], axis=1).dropna()
    df.columns = ['Product_nm','Rating','Title','Body']
        
    return df

In [11]:
samsung_df = pd.DataFrame({'Product_nm':[],
                               'Rating':[],
                                'Title':[],
                                 'Body':[],
                           'Product_cd':[]})
apple_df = samsung_df.copy()

In [39]:
# 삼성 갤럭시폰
# 데이터를 많이 추출하는 경우 예기치 못한 상황을 위하여 한 상품코드씩 파일에 저장
for i in samsung_prod_cd:
    df = amazon_reviews(i)
    if len(df) == 0:
        samsung_df_out_cd.append(i)
        pass
    else :
        samsung_df_in_cd.append(i)
        samsung_df = df
        samsung_df['Product_cd'] = i
        samsung_df['Brand'] = 'Samsung'
        
        if not os.path.exists('samsung_df.csv'):
            samsung_df.to_csv('samsung_df.csv', index=False, mode='w', encoding='utf-8-sig')
        else :
            samsung_df.to_csv('samsung_df.csv', index=False, mode='a', header=False, encoding='utf-8-sig')    

KeyboardInterrupt: 

In [27]:
# 애플 아이폰
# 데이터를 많이 추출하는 경우 예기치 못한 상황을 위하여 한 상품코드씩 파일에 저장
for i in apple_prod_cd:
    df = amazon_reviews(i)
    if len(df) == 0:
        apple_df_out_cd.append(i)
        pass
    else :
        apple_df_in_cd.append(i)
        apple_df = df
        apple_df['Product_cd'] = i
        apple_df['Brand'] = 'Apple'
        
        if not os.path.exists('apple_df.csv'):
            apple_df.to_csv('apple_df.csv', index=False, mode='w', encoding='utf-8-sig')
        else :
            apple_df.to_csv('apple_df.csv', index=False, mode='a', header=False, encoding='utf-8-sig')   

In [30]:
samsung_df.head()

,Product_nm,Rating,Title,Body,Product_cd,Brand
0,삼성 갤럭시 S21 플러스 5G SM-G996B/DS 256GB 8GB RAM 인터...,5.0,Esta hermoso pero no fue compatible con mi ope...,Esta hermoso pero no fue compatible con mi ope...,B08QXW1TXX,Samsung
1,삼성 갤럭시 S21 플러스 5G SM-G996B/DS 256GB 8GB RAM 인터...,1.0,FALSE ADVERTISEMENT,"The phone was advertised as....""This phone wil...",B08QXW1TXX,Samsung
2,삼성 갤럭시 S21 플러스 5G SM-G996B/DS 256GB 8GB RAM 인터...,4.0,"""B"" version - not compatible with AT&T",Did not have a chance to use it. Not working w...,B08QXW1TXX,Samsung
3,삼성 갤럭시 S21 플러스 5G SM-G996B/DS 256GB 8GB RAM 인터...,5.0,"So happy with the phone, as good as I expected...","I bought this for my girl, an she is so happy ...",B08QXW1TXX,Samsung
4,삼성 갤럭시 S21 플러스 5G SM-G996B/DS 256GB 8GB RAM 인터...,1.0,"This ""International"" Version is not covered by...","After 5 months, the defective screen just stop...",B08QXW1TXX,Samsung


In [31]:
apple_df.head()

,Product_nm,Rating,Title,Body,Product_cd,Brand
0,"Apple 아이폰 SE, 64GB, 레드 - 풀 언락 (프리미엄 제품)",5.0,Perfect,It was in perfect condition.,B08R988XHQ,Apple
1,"Apple 아이폰 SE, 64GB, 레드 - 풀 언락 (프리미엄 제품)",5.0,Great condition,Great condition and worked nicely as new. Woul...,B08R988XHQ,Apple
2,"Apple 아이폰 SE, 64GB, 레드 - 풀 언락 (프리미엄 제품)",5.0,Just like new,Great phone. Arrived on time. Seems just like ...,B08R988XHQ,Apple
3,"Apple 아이폰 SE, 64GB, 레드 - 풀 언락 (프리미엄 제품)",5.0,THE PHONE WAS PERFECT,Absolutely perfect!,B08R988XHQ,Apple
4,"Apple 아이폰 SE, 64GB, 레드 - 풀 언락 (프리미엄 제품)",5.0,Great Phone,Installed my sim chip and easily restored phon...,B08R988XHQ,Apple


In [32]:
smartphone = pd.concat([samsung_df,apple_df])
smartphone = smartphone.drop_duplicates()
smartphone.to_csv('smartphone.csv', index = False)

In [33]:
len(smartphone)

105637

### 모든 상품코드가 반영됐는지 검증

In [ ]:
# 삼성 갤럭시폰
print(len(samsung_prod_cd) == len(samsung_df_out_cd) + len(samsung_df_in_cd))
print('전체 코드 :', len(samsung_prod_cd) )
print('적재된 코드 : ',len(samsung_df_out_cd) + len(samsung_df_in_cd))

In [ ]:
# 애플 아이폰
len(apple_prod_cd) == len(apple_df_out_cd) + len(apple_df_in_cd)
print('전체 코드 :', len(apple_prod_cd) )
print('적재된 코드 : ',len(apple_df_out_cd) + len(apple_df_in_cd))

### csv 적재 쿼리 오류 발생

#### 주피터 노트북은 켜져 있으나 코드 실행이 취소된 경우 OR 전체코드에 비하여 적재된 코드가 적을 경우

In [38]:
# 삼성 갤럭시폰
# CSV 코드 - 적재, 미적재 코드 합치기
samsung_prod_cd_2 = samsung_df_out_cd + samsung_df_in_cd
empty_cd = []

# 모든 코드 - CSV 코드
for i in samsung_prod_cd:
    if i in samsung_prod_cd_2 :
        pass
    else :
        empty_cd.append(i)

samsung_prod_cd = empty_cd
# -> 다시 삼성 리뷰 적재 코드로

In [ ]:
# 애플 아이폰
# CSV 코드 - 적재, 미적재 코드 합치기
apple_prod_cd_2 = samsung_df_out_cd + samsung_df_in_cd
empty_cd = []

# 모든 코드 - CSV 코드
for i in apple_prod_cd:
    if i in apple_prod_cd_2 :
        pass
    else :
        empty_cd.append(i)
        
apple_prod_cd = empty_cd
# -> 다시 애플 리뷰 적재 코드로

#### 주피터 노트북이 꺼질 경우

In [26]:
# 삼성 갤럭시폰
samsung_df = pd.read_csv('samsung_df.csv', encoding = 'utf-8-sig')
samsung_prod_cd_2 = samsung_df['Product_cd']
empty_cd = []

# 모든 코드 - CSV 코드
for i in samsung_prod_cd:
    if i in samsung_prod_cd_2 :
        pass
    else :
        empty_cd.append(i)

samsung_prod_cd = empty_cd
# -> 다시 삼성 리뷰 적재 코드로

In [27]:
# 애플 아이폰
apple_df = pd.read_csv('apple_df.csv', encoding = 'utf-8-sig')
apple_prod_cd_2 = apple_df['Product_cd']
empty_cd = []

# 모든 코드 - CSV 코드
for i in apple_prod_cd:
    if i in apple_prod_cd_2 :
        pass
    else :
        empty_cd.append(i)
        
apple_prod_cd = empty_cd
# -> 다시 애플 리뷰 적재 코드로